In [ ]:
import torch
import torchvision
from google.colab import drive
drive.mount('/content/drive')
from PIL import Image
import torchvision.datasets as datasets
import torchvision.transforms.functional as TF
import matplotlib.pyplot as plt
import numpy as np

Mounted at /content/drive


In [ ]:
import numpy

def binarize_image(image_tensor):
    image_tensor[image_tensor > 0.5] = 1
    image_tensor[image_tensor <= 0.5] = 0
    return image_tensor

def get_labels_probabilities(labels_occurences):
  uniq, counts = labels_occurences.unique(return_counts=True)
  return torch.div(counts, labels_occurences.shape[0])

def generate_probabilities_matrix(train_data_tensor, labels_training, labels_probabilities, num_classes):
  complete_matrix = torch.zeros(num_classes, train_data_tensor.shape[0])
  general_probabilities_matrix = []

  for k in range(0, num_classes):
    k_type_observations = train_data_tensor[:, labels_training == k]
    k_pixels_probabilities = []
    sum = torch.sum(k_type_observations, 1)
    tensor_k_pixels_probabilities = torch.div(sum, k_type_observations.shape[1])
    complete_matrix[k].add_(tensor_k_pixels_probabilities)
  return complete_matrix

def get_probability_pixel_by_class(pixel, pixel_index, p_m_pix_val_given_k, num_class):
  matrix_given_by_pixel = p_m_pix_val_given_k[int(pixel)]
  pixel_probability_in_class = matrix_given_by_pixel[num_class][pixel_index]
  return pixel_probability_in_class


def test_model(input_torch, p_m_pix_val_given_k, p_t_tensor, num_classes = 10):
    #assumes that the input comes in a row
    k_pixels_probabilities = torch.zeros(num_classes)
    for k in range(0, num_classes):
      acum = 0.0
      for pixel_index in range(0, input_torch.shape[0]):
        probability_class_for_pixel = get_probability_pixel_by_class(input_torch[pixel_index].item(), pixel_index, p_m_pix_val_given_k, k)
        e1, e2 = torch.log(probability_class_for_pixel), torch.log(p_t_tensor[k])    # convert p1, p2 to log form
        e1e2 = e1 + e2                                                               # e1e2 is now np.log(p1 * p2)
        acum += e1e2.item()
      k_pixels_probabilities[k] = acum
    return (torch.argmax(k_pixels_probabilities).item(), k_pixels_probabilities) 

def test_model_batch(test_set, labels, p_m_pix_val_given_k, p_t_tensor):
    right_predictions = 0
    for image in range(0, test_set.shape[1]):
      (predicted_label, score_clasess) = test_model(test_set[:, image], p_m_pix_val_given_k, p_t_tensor, 10)
      if(predicted_label == labels[image]):
          right_predictions += 1
    return right_predictions / test_set.shape[1]


def train_model(train_data_tensor, labels_training, num_classes = 10):
    labels_probabilities = get_labels_probabilities(labels_training)
    matrix_probabilities_1_given_k = generate_probabilities_matrix(train_data_tensor, labels_training, labels_probabilities, num_classes)
    matrix_probabilities_0_given_k = 1 - matrix_probabilities_1_given_k
    p_m_pix_val_given_k = [matrix_probabilities_0_given_k, matrix_probabilities_1_given_k]
    return (p_m_pix_val_given_k, labels_probabilities)



def load_dataset(path = "/content/drive/MyDrive/Colab Notebooks/mnist_dataset/train"):
    #Open up the dataset
    dataset =  torchvision.datasets.ImageFolder(path)   
    list_images = dataset.imgs
    train_data_tensor  = None    
    labels_training = []
    first_tensor = True

    for i in range(0, len(list_images)):
        pair_path_label = list_images[i]        
        image = Image.open(pair_path_label[0]) 
        x_tensor = TF.to_tensor(image).squeeze()
        x_tensor_bin = binarize_image(x_tensor)       
        
        x_tensor_bin_plain = x_tensor_bin.view(x_tensor_bin.shape[0] * x_tensor_bin.shape[1], -1)

        #test dataset case        

        labels_training += [pair_path_label[1]]
        
        if(first_tensor):
            first_tensor = False
            train_data_tensor = x_tensor_bin_plain
        else:
            train_data_tensor = torch.cat((train_data_tensor, x_tensor_bin_plain), 1)
 
    return (train_data_tensor, torch.tensor(labels_training))       

    
(train_data_tensor, labels_tensor) = load_dataset(path = "/content/drive/MyDrive/Colab Notebooks/mnist_dataset/train")

p_m_pix_val_given_k , p_t_tensor = train_model(train_data_tensor, labels_tensor)

#Predecir el lable de una observacion, la observacion #500
(predicted_label, scores_classes) = test_model(train_data_tensor[:, 500], p_m_pix_val_given_k, p_t_tensor, 10)


accuracy = test_model_batch(train_data_tensor, labels_tensor, p_m_pix_val_given_k, p_t_tensor)

print("The accuracy is", accuracy)

               
    
    

The accuracy is 0.9166666666666666
